In [3]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn import metrics

In [ ]:
merged_dataset =  pd.read_csv("../data/curated/merged_dataset/2013_merged_data.parquet")

for i in [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    

In [ ]:
# holdout method to split data into train and test
num = (df_taxi_gas.count())*0.8
df_tr = df_taxi_gas.loc[:num[0], :]
df_te = df_taxi_gas.loc[num[0]:, :]

X_train = df_tr.drop(columns='avg_income_rate')
Y_train = df_tr['avg_income_rate']

y_test = df_te["avg_income_rate"]
x_test = df_te.drop(columns="avg_income_rate")

In [ ]:
# build XGBoost regression model
# https://machinelearningmastery.com/xgboost-for-regression/
model = XGBRegressor()
model.fit(X_train, Y_train)